In [2]:
import pandas as pd
import numpy as np

In [5]:
# 编码DRUG和TAR
df_d = pd.read_csv("../data/raw/Feature_DRUG.csv")
df_t = pd.read_csv("../data/raw/Feature_TAR.csv")

drug_ids = df_d['DrugID']
tar_ids = df_t['TargetID']

drug_vocab = []
for id in drug_ids:
    drug_vocab.append("Drug|||" + str(id) + "\n")

tar_vocab = []
for id in tar_ids:
    tar_vocab.append("Target|||" + str(id) + "\n")

with open("../data/processed/drug_vocab.txt", 'w') as file:
    file.writelines(drug_vocab)

with open("../data/processed/tar_vocab.txt", 'w') as file:
    file.writelines(tar_vocab)

In [6]:
with open("../data/processed/drug_vocab.txt", 'r') as file:
    drug_vocab = file.readlines()

with open("../data/processed/tar_vocab.txt", 'r') as file:
    tar_vocab = file.readlines()

drug_id_dict = {}
for i in range(len(drug_vocab)):
    drug_id_dict[drug_vocab[i][:-1]] = i

tar_id_dict = {}
for i in range(len(tar_vocab)):
    tar_id_dict[tar_vocab[i][:-1]] = i


def get_id(ID, is_drug=True):
    if is_drug:
        key = "Drug|||" + str(ID)
        return drug_id_dict[key]
    else:
        key = "Target|||" + str(ID)
        return tar_id_dict[key]


# drug_drug edge_index 和 edge_att
edge_index = []
edge_att = []
e_dd = pd.read_csv("../data/raw/Edge_DRUG-DRUG.csv", dtype='str')

cols = e_dd.columns[1:]
for index, row in e_dd.iterrows():
    id = get_id(row['DrugID'])
    for col in cols:
        if str(row['DrugID']) == str(col):
            continue
        edge_index.append([id, get_id(col)])
        edge_att.append(e_dd.loc[index, col])
with open("../data/processed/drug_drug_edge_index.txt", 'w') as file:
    for e_i in edge_index:
        file.write(str(e_i))
        file.write("\n")

with open("../data/processed/drug_drug_edge_att.txt", 'w') as file:
    for e_t in edge_att:
        file.write(str(e_t))
        file.write("\n")

In [7]:

# target_target edge_index 和 edge_att
edge_index = []
edge_att = []
e_dd = pd.read_csv("../data/raw/Edge_TAR-TAR.csv", dtype='str')

for index, row in e_dd.iterrows():
    id1 = get_id(row['TargetID1'], False)
    id2 = get_id(row['TargetID2'], False)

    edge_index.append([id1, id2])
    edge_att.append(1.0)

with open("../data/processed/tar_tar_edge_index.txt", 'w') as file:
    for e_i in edge_index:
        file.write(str(e_i))
        file.write("\n")

with open("../data/processed/tar_tar_edge_att.txt", 'w') as file:
    for e_t in edge_att:
        file.write(str(e_t))
        file.write("\n")

In [8]:

# drug_target edge_index 和 edge_att
edge_index = []
edge_att = []
e_dd = pd.read_csv("../data/raw/Edge_DRUG-TAR.csv", dtype='str')

for index, row in e_dd.iterrows():
    id1 = get_id(row['DrugID'])
    id2 = get_id(row['TargetID'], False)

    edge_index.append([id1, id2])
    edge_att.append(1.0)

with open("../data/processed/drug_tar_edge_index.txt", 'w') as file:
    for e_i in edge_index:
        file.write(str(e_i))
        file.write("\n")

with open("../data/processed/drug_tar_edge_att.txt", 'w') as file:
    for e_t in edge_att:
        file.write(str(e_t))
        file.write("\n")

In [2]:
# 数据集打乱
df = pd.read_csv("../data/raw/Label_DRUGCOMB.csv")
from sklearn.utils import shuffle

df = shuffle(df)
df.to_csv("../data/processed/Shuffled_Label_Data.csv")

In [7]:
# results.txt 转 csv
txt_file = "../data/result/GATNet(DrugA_DrugB)0--AUCs.txt"
with open(txt_file, 'r') as f:
    texts = f.readlines()

res = []
for text in texts:
    res.append(text[:-1].split('\t'))

df = pd.DataFrame(data=res)
df.to_csv("../data/result/GATNet_results.csv", header=0, index=0)

In [4]:
import random

# split to 5 folds

df = pd.read_csv("../data/raw/Label_DRUGCOMB.csv")

lenth = len(df)
pot = int(lenth / 5)
print('lenth', lenth)
print('pot', pot)
random_num = random.sample(range(0, lenth), lenth)

for i in range(5):
    test_num = random_num[pot * i:pot * (i + 1)]
    train_num = random_num[:pot * i] + random_num[pot * (i + 1):]
    test_data = df.iloc[test_num]
    train_data = df.iloc[train_num]
    test_data.to_csv("../data/processed/fold_data/fold_" + str(i + 1) + "_test.csv")
    train_data.to_csv("../data/processed/fold_data/fold_" + str(i + 1) + "_train.csv")

lenth 91155
pot 18231


In [26]:
# 标准化数据

def normalize(X, means1=None, std1=None, means2=None, std2=None, feat_filt=None, norm='tanh_norm'):
    if std1 is None:
        std1 = np.nanstd(X, axis=0)
    if feat_filt is None:
        feat_filt = std1 != 0
    X = X[:, feat_filt]
    X = np.ascontiguousarray(X)
    if means1 is None:
        means1 = np.mean(X, axis=0)
    X = (X - means1) / std1[feat_filt]
    if norm == 'norm':
        return X, (means1, std1, feat_filt)
    elif norm == 'tanh':
        return np.tanh(X), (means1, std1, feat_filt)
    elif norm == 'tanh_norm':
        X = np.tanh(X)
        if means2 is None:
            means2 = np.mean(X, axis=0)
        if std2 is None:
            std2 = np.std(X, axis=0)
        X = (X - means2) / std2
        X[:, std2 == 0] = 0
        return X, (means1, std1, means2, std2, feat_filt)


feature_cell = pd.read_csv("../data/raw/Feature_CELL.csv")
feature_tar = pd.read_csv("../data/raw/Feature_TAR.csv")
feature_drug = pd.read_csv("../data/raw/Feature_DRUG.csv")

cell = np.array(feature_cell.values)
tar = np.array(feature_tar.values)
drug = np.array(feature_drug.values)

# [122,890]
nor_cell, _ = normalize(cell[:, 1:].astype(float))
# [61,570]
nor_tar, _ = normalize(tar[:, 1:].astype(float))
# [122,188]
nor_drug, _ = normalize(drug[:, 1:].astype(float))

# _cell=np.concatenate((cell[:,0],nor_cell),axis=0)
# _tar=tar[:,0]+nor_tar
# _drug=drug[:,0]+nor_drug

cell_df = pd.DataFrame(nor_cell, index=feature_cell['Cell_Line_Name'])
tar_df = pd.DataFrame(nor_tar, index=feature_tar['TargetID'])
drug_df = pd.DataFrame(nor_drug, index=feature_drug['DrugID'])

cell_df.to_csv("../data/processed/feature/Feature_CELL.csv")
tar_df.to_csv("../data/processed/feature/Feature_TAR.csv")
drug_df.to_csv("../data/processed/feature/Feature_DRUG.csv")



C:\Users\YuJie\AppData\Local\Temp\ipykernel_17172\1268609829.py:23: RuntimeWarning: invalid value encountered in divide
  X = (X - means2) / std2


In [5]:
# 把cell feature 和 label_data 合在一起
cells = pd.read_csv("../data/processed/feature/Feature_CELL.csv", index_col='Cell_Line_Name', dtype=str)
label_data = pd.read_csv("../data/raw/Label_DRUGCOMB.csv")
cell_list = list(label_data['Cell_Line_Name'].str.lower())

cells.index = cells.index.str.lower()

features_cell_np = cells.loc[cell_list].to_numpy().astype('float')

label_data_np = label_data.to_numpy()
final = np.concatenate((label_data_np, features_cell_np), axis=1)

cell_col = ["cell_feature_" + str(x) for x in cells.columns]




In [8]:
new_col = list(label_data.columns)
new_col.extend(cell_col)

res = pd.DataFrame(final, columns=new_col)
res.to_csv("../data/processed/all_data.csv")